In [1]:
!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/minsearch.py

--2024-06-11 08:28:27--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-06-11 08:28:27 (14.7 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [4]:
!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json

--2024-06-11 08:39:06--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 658332 (643K) [text/plain]
Saving to: ‘documents.json’

documents.json      100%[===================>] 642.90K  --.-KB/s    in 0.01s   

2024-06-11 08:39:07 (56.2 MB/s) - ‘documents.json’ saved [658332/658332]



In [2]:
import minsearch

In [3]:
import json

In [7]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [9]:
#docs_raw

documents = []
for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [11]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [12]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [14]:
q ='the course has already started, can i still enroll?'

In [15]:
index.fit(documents)

In [20]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [21]:
import openai

from openai import OpenAI

client = OpenAI()

In [22]:
q

'the course has already started, can i still enroll?'

In [24]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": q}]
)    

In [28]:
response.choices[0].message.content

"It depends on the policies of the institution offering the course. Here are a few steps you could take:\n\n1. **Check the Registration Deadline**: Look up the course information on the institution's website or contact the registrar's office to find out if late enrollment is allowed.\n\n2. **Contact the Instructor**: Sometimes, the course instructor has the discretion to allow late enrollments. Send an email or visit during office hours to explain your situation and express your interest in joining the class.\n\n3. **Speak with an Academic Advisor**: They can provide guidance on the institution's policies regarding late enrollment and may assist you in the process.\n\n4. **Review Course Requirements**: Understand any missed material or assignments to ensure you can catch up if you are allowed to enroll.\n\n5. **Complete Necessary Paperwork**: If late enrollment is permitted, you may need to fill out additional forms or get special permissions. Be prepared to act quickly to meet any new

In [38]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.
if the CONTEXT doesn't contain the answer, output NONE

QUESTION: the course has already started, can i still enroll?

CONTEXT: 
section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your fi

"Yes, you can still enroll in the course after it has started. Even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects, so don't leave everything for the last minute."

In [71]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
    query=query,
    filter_dict={'course': 'data-engineering-zoomcamp'},
    boost_dict=boost,
    num_results=5
)
    return results

In [72]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
   
    
    QUESTION: {question}
    
    CONTEXT: 
    {context}
    """.strip()
    
    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [74]:
def llm(prompt):
    response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": prompt}]
)

    return response.choices[0].message.content

In [75]:
query = 'how do i run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [76]:
rag(query)

'To run Kafka producer/consumer/KStreams in terminal for a Java project, follow these steps:\n\n1. Navigate to your project directory.\n2. Run the following command:\n    ```sh\n    java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n    ```\n\nReplace `<jar_name>` with the name of your JAR file.\n\nIf you are using Python and encounter the "Module \'kafka\' not found" error when trying to run `producer.py`, you should:\n\n1. Create a virtual environment:\n    ```sh\n    python -m venv env\n    ```\n\n2. Activate the virtual environment:\n    - On MacOS/Linux:\n        ```sh\n        source env/bin/activate\n        ```\n    - On Windows:\n        ```sh\n        env\\Scripts\\activate\n        ```\n\n3. Install the required packages:\n    ```sh\n    pip install -r ../requirements.txt\n    ```\n\n4. Deactivate the virtual environment when you are done:\n    ```sh\n    deactivate\n    ```\n\nEnsure Docker images are up and running before activ

In [78]:
rag('the course has already started, can i still enroll?')

'Yes, you can still enroll in the course even after it has started. You are eligible to submit the homeworks, but be mindful of the deadlines for turning in the final projects. Make sure not to leave everything until the last minute.'